# Emulation of nonlinear audio systems with LSTM in Real Time
version : model 4 with parameters

Imports :

In [1]:
import sys,select,os
sys.path.append('./Codes')
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle
import scipy.io.wavfile
import time
import pyaudio 
from tensorflow.python.client import timeline
from tensorflow.contrib.rnn import *




pathModel = './Experiments/2019-05-20-17-21_model4/temp/' #(Nh=150,N=100) model 4






if not os.path.isdir(pathModel):
    print("could nt find path model {}".format(pathModel))
num_step = 100
bufferAudioSize = 400
fs = 22050


with tf.Session() as sess:
    #config=tf.ConfigProto(log_device_placement=True)
    saver = tf.train.import_meta_graph(pathModel+'myBestModel.ckpt.meta')
    saver.restore(sess,tf.train.latest_checkpoint(pathModel))
    lastBuff = np.zeros((num_step-1,))
    graph = tf.get_default_graph()
    data = graph.get_tensor_by_name("placeHolder/data:0")
    #is_training = graph.get_tensor_by_name("placeHolder/is_training:0")
    #prediction = graph.get_tensor_by_name("FCLayer/prediction:0")
    prediction = graph.get_tensor_by_name("FCPred/Tanh:0")
    
    
    #options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    #run_metadata = tf.RunMetadata()
    
    dataNonShaped = tf.placeholder(tf.float32, [bufferAudioSize,],name="inputFromADC")
    
    prevBuff = tf.placeholder(tf.float32, [num_step-1,],name = "EndofPreviousInputBufferNonShaped")
    #with tf.device('/cpu:0'):
    newBuff = tf.concat([prevBuff,dataNonShaped],0) #size num_step-1+bufferAudioSize
    nextBuff = newBuff[-(num_step-1):] # save for next iteration
    
    my_indices = tf.constant(np.arange(bufferAudioSize))
    indices = (np.arange(num_step) +my_indices[:,tf.newaxis])
    matrixParameters = tf.constant(5.0, shape=[bufferAudioSize, num_step])
    #dataShaped = tf.gather(newBuff,indices) # slice input vector into tensor of shape(audiobufferSize,num_step)
    dataShaped = tf.stack((tf.gather(newBuff,indices),matrixParameters),axis=2)
    #print(dataShaped.get_shape())
    r=0
    timeTot=0
    def callback(in_data, frame_count, time_info, flag):
        if flag:
            print("Playback Error: {}".format(flag))
        global lastBuff
        global r
        global timeTot
        tstart = time.time()
        audio_data = np.frombuffer(in_data, dtype=np.float32)
        
        dataShapedToProcess,lastBuff = sess.run([dataShaped,nextBuff], feed_dict={dataNonShaped : audio_data, prevBuff : lastBuff})
        #print(np.shape(dataShapedToProcess))
        
        #out = sess.run(prediction, feed_dict={data: dataShapedToProcess}, options=options, run_metadata=run_metadata)
        out = sess.run(prediction, feed_dict={data: dataShapedToProcess})
        tstop = time.time()
        timeIt = tstop-tstart
        timeTot+=timeIt
        r+=1
        if(r % 100==0):
            print("out:{}".format(timeTot/r))
            r=0
            timeTot=0
        
            # Create the Timeline object, and write it to a json file
        #fetched_timeline = timeline.Timeline(run_metadata.step_stats)
        #chrome_trace = fetched_timeline.generate_chrome_trace_format()
        #with open('timeline_01.json', 'w') as f:
        #    f.write(chrome_trace)
        return out, pyaudio.paContinue
    
    #fake graph use before launch audio stream, to place node on the graph
    for x in range(0, 5):
        initInput = [0]*bufferAudioSize
        dataShapedToProcess = sess.run([dataShaped], feed_dict={dataNonShaped : initInput, prevBuff : lastBuff})
        _ = sess.run(prediction, feed_dict={data: dataShapedToProcess[0]})
    
    pa = pyaudio.PyAudio()
    
    stream = pa.open(format = pyaudio.paFloat32,
                     channels = 1,
                     rate = fs,
                     output = True,
                     input = True,
                     frames_per_buffer = bufferAudioSize,
                     stream_callback = callback)
    print("input latency {} s".format(stream.get_input_latency()))
    print("output latency {} s".format(stream.get_output_latency()))

    #stream.start_stream()
    while stream.is_active():   
        time.sleep(0.1)

    print("stream interrupted")   
    stream.stop_stream()
    stream.close()
    pa.terminate()

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./Experiments/2019-05-20-17-21_model4/temp/myFinalModel.ckpt
input latency 0.036281179138321996 s
output latency 0.054421768707483 s
out:0.006677951812744141
out:0.00649885892868042
out:0.0065053534507751466
out:0.006496317386627197
out:0.006281452178955078
out:0.0066208171844482425
out:0.0065228652954101565
out:0.006491756439208985
out:0.006525945663452148
out:0.0063057255744934084


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-fc30bda2c5d3>", line 111, in <module>
    time.sleep(0.1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 1828, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/ultratb.py", line 1090, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/ultratb.py", line 311, in wrapp

KeyboardInterrupt: 